<a href="https://colab.research.google.com/github/pea-sys/Quevico/blob/master/Predicting_Engineer_Salaries__LGBM%2BCAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#StackOverFlowのアンケート内容から回答者の給与を予測@Quevico
順位：2位/256人(暫定)  
環境：所有PCがタブレットしかないため、GoogleColabで実装しています。  
モデル：重要な特徴量を足し引きしたLGBMモデル3個とCatboostモデル3個のアンサンブル  
工夫した点：


*   未解答欄の多いユーザのweightを下げる
*   フルトレイン
*   年収に寄与しない特徴の加工(未就業者のスキルセットをnanで埋める等)  
*   交互作用項の作りこみ

感想：  
３万円とはいえ、初めて賞金圏を取ったコンペになった。  
今回初めてCatBoostを使用したが、カテゴリ値が多くデータがやや少ないものに適していると感じた。


In [ ]:
!nvidia-smi

Sat Sep  5 21:00:09 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install category_encoders
!pip install -U catboost

Requirement already up-to-date: catboost in /usr/local/lib/python3.6/dist-packages (0.24.1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd /content/drive/My\ Drive/quevico

/content/drive/My Drive/quevico


In [ ]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, random, pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from math import sqrt
import lightgbm as lgb
import catboost as cat
from catboost import CatBoostRegressor, Pool, cv
import category_encoders as ce

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 129)
pd.set_option("display.max_colwidth", 500)
warnings.filterwarnings('ignore')

In [ ]:
SUB_FILE = "submit.csv"
TEST_FILE = "test.csv"
TRAIN_FILE= "train.csv"
QUESTION_FILE = "question.csv"
TARGET = "Salary"

In [ ]:
train_data = pd.read_csv(TRAIN_FILE)
train_data = train_data.set_index('No')
test_data = pd.read_csv(TEST_FILE)
test_data[TARGET] = 0
test_data = test_data.set_index('No')
print(train_data.shape,test_data.shape)

(33857, 127) (11259, 127)


#Preprocess

In [ ]:
def add_nan_count_and_clean(df):
  print('add_nan_count start',df.shape)
  df['nanCnt'] = df.isnull().sum(axis=1)
  df['nanCnt'] = df['nanCnt'].astype(np.int16)
  
  df['HypotheticalTools_Total'] = 0
  c = 'HypotheticalTools'
  for i in range(1,6):
    df[c+str(i)] = df[c+str(i)].astype(np.object)	
    df.loc[df[c+str(i)]=='Not at all interested','HypotheticalTools_Total'] +=  np.sqrt(1)
    df.loc[df[c+str(i)]=='Somewhat interested','HypotheticalTools_Total'] += np.sqrt(3)
    df.loc[df[c+str(i)]=='A little bit interested','HypotheticalTools_Total'] += np.sqrt(4)
    df.loc[df[c+str(i)]=='Very interested','HypotheticalTools_Total'] += np.sqrt(5)
    df.loc[df[c+str(i)]=='Extremely interested','HypotheticalTools_Total'] += np.sqrt(6)
    del df[c+str(i)]
  df['HypotheticalTools_Total'] = df['HypotheticalTools_Total'].astype(np.int8)
  #趣味でプログラミングやっていない場合
  df.loc[df['Hobby']== 'No','HoursComputer'] = 'Less than 1 hour'
  df.loc[(df['LastNewJob']==r"I've never had a job") & (df['DevType'].astype(str).str.contains("Student")),'DevType'] = "Student"
  df.loc[df['LastNewJob']==r"I've never had a job",'CommunicationTools'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'LanguageWorkedWith'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'LanguageDesireNextYear'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'Methodology'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'FrameworkDesireNextYear'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'FrameworkWorkedWith'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'PlatformDesireNextYear'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'PlatformWorkedWith'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'DatabaseDesireNextYear'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'DatabaseWorkedWith'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'HackathonReasons'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'SelfTaughtTypes'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'EducationTypes'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'IDE'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'VersionControl'] = np.nan
  df.loc[(df['LastNewJob']==r"I've never had a job") & (df['Country']=='United States'),'MilitaryUS'] = 'No'
  df.loc[df['LastNewJob']==r"I've never had a job",'CompanySize'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'YearsCodingProf'] = np.nan
  df.loc[df['LastNewJob']==r"I've never had a job",'Employment'] = 'Not employed, and not looking for work'
  #TRAINのみ(暫定)
  if test_data[TARGET].sum() != 0:
    print("train")
    df = df.loc[~((df['LastNewJob']==r"I've never had a job") & (df['Age'] == '35 - 44 years old'))]
    df.loc[(df['LastNewJob']==r"I've never had a job") & (df[TARGET]>=50000) ,'LastNewJob'] = np.nan
  print('add_nan_count end',df.shape)
  return df

In [ ]:
def preprocess_feature_split(df):
  print('preprocess_feature_split start',df.shape)
  split_features = ['DevType','CommunicationTools','HackathonReasons','LanguageWorkedWith','DatabaseWorkedWith','PlatformWorkedWith','VersionControl','RaceEthnicity']
  split_features += ['EducationTypes','SelfTaughtTypes','FrameworkWorkedWith','FrameworkDesireNextYear','IDE','Methodology','AdsActions','ErgonomicDevices']
  split_features += ['SexualOrientation','LanguageDesireNextYear','PlatformDesireNextYear','DatabaseDesireNextYear']
  split_features += ['AdBlockerReasons','Gender']

  for s in split_features:
    df[s] = df[s].str.replace('Visual Studio Code','VSCode',regex = True)
    df[s] = df[s].str.replace(r'(','',regex = True) #括弧で囲まれていると変換できない
    df[s] = df[s].str.replace(r')','',regex = True)
    df[s] = df[s].str.replace(r'+','p',regex = True)
    df[s] = df[s].str.replace(r'+','p',regex = True)
    df[s] = df[s].str.replace(r'[','p',regex = True)
    df[s] = df[s].str.replace(r']','p',regex = True)
    df[s] = df[s].str.replace(r'{','p',regex = True)
    df[s] = df[s].str.replace(r'}','p',regex = True)
    df[s] = df[s].str.replace(r",",'p',regex = True)

    df[s+'Count'] = df[s].astype(str).apply(lambda x: len(x.split(";")))
    df[s+'Count'] = df[s+'Count'].astype(np.int8)
    df[s].fillna('Nothing',inplace=True)
    l = list()
    for r in df[s].str.split(';').values:
      l.extend(r) 
    l = list(set(l))

    for o in l:
      df[s + "_" + o] = df[r""+ s].str.contains(r"" + o)
    del df[s]
    del df[s+"_Nothing"]
  print('preprocess_feature_split end',df.shape)
  return df

In [ ]:
def preprocess_other(df):
  print('preprocess start',df.shape)
  #プロ経験がないため、倫理観に信頼性なし
  for c in ['EthicsChoice','EthicsReport','EthicsResponsible','EthicalImplications']:
    df.loc[df['YearsCodingProf'].isnull(),c] = np.nan

  #コーディング経験の矛盾修正
  df.loc[df['YearsCoding'] < df['YearsCodingProf'],'YearsCodingProf'] = df['YearsCoding']

  #NaN補完
  df.loc[df['YearsCoding'].isnull(),'YearsCoding'] = df['YearsCodingProf']
  df = df.loc[~df['JobSearchStatus'].isnull()]

  df.loc[(df['SelfTaughtTypes_A book or e-book from O’Reillyp Apressp or a similar publisher']) & (df['Age']<=24),'SelfTaughtTypes_A book or e-book from O’Reillyp Apressp or a similar publisher'] = False
  df.loc[(df['EducationTypes_Participated in online coding competitions e.g. HackerRankp CodeChefp TopCoder']) & (df['Age']>=44),'EducationTypes_Participated in online coding competitions e.g. HackerRankp CodeChefp TopCoder'] = False
  df.loc[(df['EducationTypes_Completed an industry certification program e.g. MCPD']) & (df['Age']<=24),'EducationTypes_Completed an industry certification program e.g. MCPD'] = False
  print('preprocess end',df.shape)
  return df

In [ ]:
def feature_encoder(df):
  print("feature_encoder start")
  for c in ['YearsCoding','YearsCodingProf']:
    df.loc[df[c]=='0-2 years',c] = 1
    df.loc[df[c]=='3-5 years',c] = 4
    df.loc[df[c]=='6-8 years',c] = 7
    df.loc[df[c]=='9-11 years',c] = 10
    df.loc[df[c]=='12-14 years',c] = 13
    df.loc[df[c]=='15-17 years',c] = 16
    df.loc[df[c]=='18-20 years',c] = 19
    df.loc[df[c]=='21-23 years',c] = 22
    df.loc[df[c]=='24-26 years',c] = 25
    df.loc[df[c]=='27-29 years',c] = 28
    df.loc[df[c]=='30 or more years',c] = 31
    df[c] = df[c].astype(np.float32)

  c = 'Age'
  df.loc[df[c]=='Under 18 years old',c] = 18
  df.loc[df[c]=='18 - 24 years old',c] = 24
  df.loc[df[c]=='25 - 34 years old',c] = 34
  df.loc[df[c]=='35 - 44 years old',c] = 44
  df.loc[df[c]=='45 - 54 years old',c] = 54
  df.loc[df[c]=='55 - 64 years old',c] = 64
  df.loc[df[c]=='65 years or older',c] = 74
  df[c] = df[c].astype(np.float32)
  
  c = 'Student'
  df.loc[df[c]=='Yes, full-time',c] = 8
  df.loc[df[c]=='Yes, part-time',c] = 4
  df.loc[df[c]=='No',c] = 0
  df[c] = df[c].astype(np.float16)

  c = 'CompanySize'
  df.loc[df[c]=='Fewer than 10 employees',c] = np.sqrt(9) 
  df.loc[df[c]=='10 to 19 employees',c] = np.sqrt(19)
  df.loc[df[c]=='20 to 99 employees',c] = np.sqrt(99)
  df.loc[df[c]=='100 to 499 employees',c] = np.sqrt(499)
  df.loc[df[c]=='500 to 999 employees',c] = np.sqrt(999)
  df.loc[df[c]=='1,000 to 4,999 employees',c] = np.sqrt(4999)
  df.loc[df[c]=='5,000 to 9,999 employees',c] = np.sqrt(9999)
  df.loc[df[c]=='10,000 or more employees',c] = np.sqrt(10000)
  df[c] = df[c].astype(np.float32)

  c = 'Employment'
  df.loc[df[c]=='Employed full-time',c] = 8
  df.loc[df[c]=='Independent contractor, freelancer, or self-employed',c] = 8
  df.loc[df[c]=='Employed part-time',c] = 4
  df.loc[df[c]=='Not employed, but looking for work',c] = 0
  df.loc[df[c]=='Not employed, and not looking for work',c] = 0
  df.loc[df[c]=='Retired',c] = 0
  df[c] = df[c].astype(np.float16)

  c = 'HoursOutside'
  df.loc[df[c]=='Less than 30 minutes',c] = 30 
  df.loc[df[c]=='30 - 59 minutes',c] = 59 
  df.loc[df[c]=='1 - 2 hours',c] = 120
  df.loc[df[c]=='3 - 4 hours',c] = 240
  df.loc[df[c]=='Over 4 hours',c] = 300
  df[c].fillna(-999,inplace=True)
  df[c] = df[c].astype(np.int16)

  c = 'HoursComputer'
  df.loc[df[c]=='Less than 1 hour',c] =  np.sqrt(1)
  df.loc[df[c]=='1 - 4 hours',c] =  np.sqrt(4)
  df.loc[df[c]=='5 - 8 hours',c] =  np.sqrt(8)
  df.loc[df[c]=='9 - 12 hours',c] =  np.sqrt(12)
  df.loc[df[c]=='Over 12 hours',c] =  np.sqrt(16)
  df[c] = df[c].astype(np.float16)

  c = 'SkipMeals'
  df.loc[df[c]=='Never',c] = 0
  df.loc[df[c]=='Daily or almost every day',c]=np.sqrt(1) #規則正しい
  df.loc[df[c]=='1 - 2 times per week',c] = np.sqrt(2)
  df.loc[df[c]=='3 - 4 times per week',c] = np.sqrt(4)
  df[c] = df[c].astype(np.float16)
                         
  c = 'Exercise'
  df.loc[df[c]=='I don\'t typically exercise',c] = 0
  df.loc[df[c]=='Daily or almost every day',c]=1 #規則正しい
  df.loc[df[c]=='1 - 2 times per week',c] = 2
  df.loc[df[c]=='3 - 4 times per week',c] = 4
  df[c] = df[c].astype(np.float16)

  c = 'TimeFullyProductive'
  df.loc[df[c]=='Less than a month',c] = np.sqrt(0.5)
  df.loc[df[c]=='One to three months',c] = np.sqrt(1)
  df.loc[df[c]=='Three to six months',c] = np.sqrt(3)
  df.loc[df[c]=='Six to nine months',c] = np.sqrt(6)
  df.loc[df[c]=='Nine months to a year',c] = np.sqrt(9)
  df.loc[df[c]=='More than a year',c]= np.sqrt(12)
  df[c] = df[c].astype(np.float32)
  c = 'AssessJob'
  for i in range(1,11):
    df[c+str(i)].fillna(-999,inplace=True)
    df[c+str(i)] = df[c+str(i)].astype(np.int16)
  c ='AssessBenefits'
  for i in range(1,12):
    df[c+str(i)].fillna(-999,inplace=True)
    df[c+str(i)] = df[c+str(i)].astype(np.int16)
  c='JobContactPriorities'
  for i in range(1,6):
    df[c+str(i)].fillna(-999,inplace=True)
    df[c+str(i)] = df[c+str(i)].astype(np.int16)
  c='AdsPriorities'
  for i in range(1,8):
    df[c+str(i)].fillna(-999,inplace=True)
    df[c+str(i)] = df[c+str(i)].astype(np.int16)
  c='JobEmailPriorities'
  for i in range(1,3):
    df[c+str(i)].fillna(-999,inplace=True)
    df[c+str(i)] = df[c+str(i)].astype(np.int16)
  print("feature_encoder end")
  return df

In [ ]:
def feature_add(tr,ts):
  print("feature_add start",tr.shape,ts.shape)
  
  tmp = pd.concat([tr.loc[~tr['YearsCodingProf'].isnull()],ts[~ts['YearsCodingProf'].isnull()]])[['Country','YearsCodingProf']].groupby('Country').mean().reset_index()
  tmp.rename(columns={'YearsCodingProf':'Country_YearsCodingProf_Mean'},inplace=True)
  tr = tr.merge(tmp,on=['Country'],how='left')
  ts = ts.merge(tmp,on=['Country'],how='left')

  tmp = pd.concat([tr.loc[~tr['Age'].isnull()],ts[~ts['Age'].isnull()]])[['Country','Age']].groupby('Country').mean().reset_index()
  tmp.rename(columns={'Age':'Country_Age_Mean'},inplace=True)
  tr = tr.merge(tmp,on=['Country'],how='left')
  ts = ts.merge(tmp,on=['Country'],how='left')

  tmp = pd.concat([tr.loc[~tr['Age'].isnull()],ts[~ts['Age'].isnull()]])[['Country','Age']].groupby('Country').std().reset_index()
  tmp.rename(columns={'Age':'Country_Age_Std'},inplace=True)
  tr = tr.merge(tmp,on=['Country'],how='left')
  ts = ts.merge(tmp,on=['Country'],how='left')

  tmp = pd.concat([tr.loc[(~tr['Age'].isnull()) & (~tr['Student'].isnull())],ts[(~ts['Age'].isnull()) & (~ts['Student'].isnull()) ]])[['Country','Student','Age']].groupby(['Country','Student']).mean().reset_index()
  tmp.rename(columns={'Age':'Country_Student_Age_Mean'},inplace=True)
  tr = tr.merge(tmp,on=['Country','Student'],how='left')
  ts = ts.merge(tmp,on=['Country','Student'],how='left')
  
  tmp = pd.concat([tr.loc[(~tr['HoursComputer'].isnull())],ts[(~ts['HoursComputer'].isnull())]])[['Age','HoursComputer']].groupby(['Age']).mean().reset_index()
  tmp.rename(columns={'HoursComputer':'Age_HoursComputer_Mean'},inplace=True)
  tr = tr.merge(tmp,on=['Age'],how='left')
  ts = ts.merge(tmp,on=['Age'],how='left')

  tmp = pd.concat([tr.loc[(~tr['HoursOutside'].isnull())],ts[(~ts['HoursOutside'].isnull())]])[['Age','HoursOutside']].groupby(['Age']).mean().reset_index()
  tmp.rename(columns={'HoursOutside':'Age_HoursOutside_Mean'},inplace=True)
  tr = tr.merge(tmp,on=['Age'],how='left')
  ts = ts.merge(tmp,on=['Age'],how='left')

  tmp = pd.concat([tr,ts])[['Country','Hobby']].groupby('Country').count().reset_index()
  tmp.rename(columns={'Hobby':'Country_Count'},inplace=True)
  tmp['Country_Count'] = tmp['Country_Count'].astype(np.int32)
  tr = tr.merge(tmp,on='Country',how='left')
  ts = ts.merge(tmp,on='Country',how='left')

  for c in ['Country','CompanySize','Age','LastNewJob','NumberMonitors','JobSatisfaction']:
    tmp = tr[[c,TARGET]].groupby(c).mean().reset_index()
    tmp.rename(columns={TARGET:c+'_Salary_Mean'},inplace=True)
    tr = tr.merge(tmp,on=c,how='left')
    ts = ts.merge(tmp,on=c,how='left')
  
  tmp = tr[['Employment','SalaryType',TARGET]].groupby(['Employment','SalaryType']).mean().reset_index()
  tmp.rename(columns={TARGET:'Employment_SalaryType_Salary_Mean'},inplace=True)
  tr = tr.merge(tmp,on=['Employment','SalaryType'],how='left')
  ts = ts.merge(tmp,on=['Employment','SalaryType'],how='left')
  print("feature_add end",tr.shape,ts.shape)
  return tr,ts

In [ ]:
def remove_features(tr,ts):
  print("remove_features start",tr.shape,ts.shape)
  #データクリーニング
  tr = tr.loc[~(tr['FrameworkWorkedWithCount'] >10)]
  tr = tr.loc[~((tr['Age']==18) & (tr['HopeFiveYears']=='Retirement'))]
  tr = tr.loc[tr['LanguageWorkedWithCount']<38]

  remove_col = ['Currency','SexualOrientationCount','PlatformWorkedWithCount','PlatformDesireNextYearCount','SelfTaughtTypesCount','IDECount','LanguageDesireNextYearCount','DatabaseDesireNextYearCount','SurveyTooLong']#,'Country_Count'
  remove_col += ['FrameworkWorkedWithCount','FrameworkDesireNextYearCount','RaceEthnicityCount','DatabaseWorkedWithCount','MethodologyCount','CommunicationToolsCount','ErgonomicDevicesCount',]#,'nanCnt','HackathonReasonsCount'
  remove_col += ['EducationParents','StackOverflowDevStory','StackOverflowJobsRecommend','CommunicationTools_Facebook','SelfTaughtTypes_A college/university computer science or software engineering book']#'EducationTypes_Taken an online course in programming or software development e.g. a MOOC'
  remove_col += ['SelfTaughtTypes_Internal Wikisp chat roomsp or documentation set up by my company for employees']
  remove_col += ['HackathonReasons_To improve my ability to work on a team with other programmers','EducationTypes_Participated in a full-time developer training program or bootcamp','IDE_Zend']
  remove_col += ['AdsActions_Saw an online advertisement and then researched it without clicking on the ad','VersionControlCount','VersionControl_Mercurial','IDE_Light Table']
  for c in ['JobEmailPriorities']:
    for i in range(3,8):
      remove_col.append(c+str(i))
  for c in ['AdsPriorities']:
    for i in range(1,5):
      remove_col.append(c+str(i))
  for c in remove_col:
    del tr[c],ts[c]

  print("remove_features end",tr.shape,ts.shape)
  return tr,ts

In [ ]:
def get_split_column(col,df):
  cols =[]
  for c in df.columns:
    if col in c:
      cols.append(c)
  return cols

def column_compaction(df,col_prefix):
  print(col_prefix+'_column_compaction start',df.shape)
  p = col_prefix+"WorkedWith_"
  for c in get_split_column(p,df):
    nc  = c.replace(p,"")
    df[nc] = df[c].astype(np.int8)
    del df[c]

  p = col_prefix+"DesireNextYear_"
  for c in get_split_column(p,df):
    nc  = c.replace(p,"")
    df[nc] += df[c].astype(np.int8)
    del df[c]
  print(col_prefix+'_column_compaction end',df.shape)
  return df

def interaction_feature(df,features,newcolumn=""):
  if len(newcolumn) > 0 :
    df[newcolumn] = ""

  df[features[0]] = df[features[0]].astype(str)
  for i in range(1,len(features)):
    df[features[0]] += df[features[i]].astype(str)
    del df[features[i]]

  if len(newcolumn) > 0 :
    df[newcolumn] = df[features[0]] 
    del df[features[0]]
  return df

In [ ]:
def preprocess_interaction(df):
  df.loc[df['Country']=='United States','Country'] = df['Country'].astype(str) + df['MilitaryUS'].astype(str)
  del df['MilitaryUS']
  df = interaction_feature(df,['DevType_Data scientist or machine learning specialist','EducationTypes_Taken an online course in programming or software development e.g. a MOOC'])#TRY
  df = interaction_feature(df,['CheckInCode','Hobby'])
  df = interaction_feature(df,['Student','DevType_Student'])
  df = interaction_feature(df,['Python','IDE_IPython / Jupyter','IDE_PyCharm'])
  df = interaction_feature(df,['Java','IDE_Eclipse','IDE_NetBeans'])
  df = interaction_feature(df,['Ruby','IDE_RubyMine'])
  df = interaction_feature(df,['R','IDE_RStudio'])
  df = interaction_feature(df,['FormalEducation','UndergradMajor'])
  df = interaction_feature(df,['EducationTypes_Participated in a hackathon','HackathonReasonsCount'])
  df = interaction_feature(df,['MongoDB','Heroku'])
  df = interaction_feature(df,['Drupal','WordPress'],'Drupal_WordPress')
  df = interaction_feature(df,['Google Home','Amazon Echo'],'SmartDevice')
  df = interaction_feature(df,['EducationTypes_Taken a part-time in-person course in programming or software development','SelfTaughtTypes_Pre-scheduled tutoring or mentoring sessions with a friend or colleague'])
  df = interaction_feature(df,['EducationTypes_Participated in online coding competitions e.g. HackerRankp CodeChefp TopCoder','HackathonReasons_To help me find new job opportunities'])
  df = interaction_feature(df,['DevType_Engineering manager','DevType_Product manager','DevType_C-suite executive CEOp CTOp etc.'],'DevType_Manager')
  df = interaction_feature(df,['DevType_Data or business analyst','DevType_Data scientist or machine learning specialist','DevType_Marketing or sales professional'],'DevType_DataScientist')
  df = interaction_feature(df,['DevType_Designer','DevType_Game or graphics developer','DevType_Front-end developer'],'DevType_Designer')
  df = interaction_feature(df,['DevType_Database administrator','DevType_System administrator'],'DevType_Admin')
  df = interaction_feature(df,['DevType_Desktop or enterprise applications developer','DevType_Embedded applications or devices developer','DevType_Mobile developer'],'DevType_Developer')
  df = interaction_feature(df,['DevType_QA or test developer','DevType_Back-end developer','DevType_DevOps specialist'],'DevType_Infra')
  df = interaction_feature(df,['DevType_Full-stack developer','DevType_Educator or academic researcher'],'DevType_Other')
  return df

In [ ]:
def set_weight(data):
  data['weight'] = 1
  data.loc[data['nanCnt'].astype(np.float)>=100,'weight'] = 0.8
  del data['nanCnt']
  return data

def columns_sort(tr,ts):
  tr['data'] = 'train'
  ts['data'] ='test'

  data = pd.concat([tr,ts])
  tr = data.loc[data['data']=='train'].drop('data',axis=1)
  ts = data.loc[data['data']=='test'].drop('data',axis=1)
  del data
  return tr,ts

In [ ]:
def preprocess(tr,ts):
  for df in [tr,ts]:
    df = add_nan_count_and_clean(df)
    df = preprocess_feature_split(df)
    df = feature_encoder(df)

  tr,ts = feature_add(tr,ts)

  tr = preprocess_other(tr)
  ts = preprocess_other(ts)

  tr,ts = remove_features(tr,ts)

  for c in ['Database','Framework','Platform','Language']:
    tr = column_compaction(tr,c)
    ts = column_compaction(ts,c)
  
  tr =  preprocess_interaction(tr)
  ts =  preprocess_interaction(ts)
  
  tr = set_weight(tr)
  ts = set_weight(ts)

  tr,ts = columns_sort(tr,ts)
  tr.to_csv("preprocess_"+TRAIN_FILE)
  ts.to_csv("preprocess_"+TEST_FILE)
  return tr,ts

In [ ]:

def train():
  #パラメータチューニング時のみバリデーションデータ作成。提出ファイル作成時は訓練データをすべて訓練に使用。
  #CatBoostがRAMの都合上、lr高めiteration低め
  cat_params = {
      'learning_rate': 0.05,
      'eval_metric': 'RMSE',
      'random_seed': 1205,
      'use_best_model': False,
      'verbose': 500,
      'num_boost_round':7000,
      'od_type': 'Iter',
      'od_wait': 300,
      'one_hot_max_size':3}
      
  lgb_params = {
        'task' : 'train',
        'boosting_type' : 'dart',
        'objective' : 'regression',
        'metric' : 'rmse',
        'num_leaves' : 50,
        'learning_rate' : 0.01,
        'feature_fraction' : 0.8,
        'bagging_fraction' : 0.8,
        'bagging_freq': 5}
  #lgb 3モデル catboost 3モデル作成
  for d in ["","Country","Country_Count"]:
    train_data = pd.read_csv("preprocess_"+TRAIN_FILE,index_col=0)
    test_data = pd.read_csv("preprocess_"+TEST_FILE,index_col=0)

    if d in train_data.columns:
      del train_data[d],test_data[d]
    
    categorical_features_indices = []
    df = train_data.drop(TARGET, axis=1)
    for c in range(0,len(df.columns)):
      if 'float' not in str(df[df.columns[c]].dtype):
        categorical_features_indices.append(df.columns[c])
        train_data[df.columns[c]].fillna(-999,inplace=True)
        test_data[df.columns[c]].fillna(-999,inplace=True)
        train_data[df.columns[c]] =  train_data[df.columns[c]].astype('category')
        test_data[df.columns[c]] =  test_data[df.columns[c]].astype('category')
    
    model = CatBoostRegressor(**cat_params)
    cat_train = Pool(train_data.drop(TARGET,axis=1).drop('weight',axis=1), train_data[TARGET], cat_features=categorical_features_indices)
    model.fit(cat_train)
    model.save_model('cat_' + d)
    del model
    
    lgb_train = lgb.Dataset(train_data.drop(TARGET,axis=1).drop('weight',axis=1), train_data[TARGET],weight = train_data['weight'])
    lgb_train.free_raw_data = None
    model = lgb.train(lgb_params,
            lgb_train,
            num_boost_round=27000,
            categorical_feature = categorical_features_indices)
  
    model.save_model('lgb_' + d)


In [ ]:
train_data,test_data = preprocess(train_data,test_data)

add_nan_count start (33857, 127)
add_nan_count end (33857, 124)
preprocess_feature_split start (33857, 124)
preprocess_feature_split end (33857, 444)
feature_encoder start
feature_encoder end
add_nan_count start (11259, 127)
add_nan_count end (11259, 124)
preprocess_feature_split start (11259, 124)
preprocess_feature_split end (11259, 444)
feature_encoder start
feature_encoder end
feature_add start (33857, 444) (11259, 444)
feature_add end (33857, 458) (11259, 458)
preprocess start (33857, 458)
preprocess end (33856, 458)
preprocess start (11259, 458)
preprocess end (11259, 458)
remove_features start (33856, 458) (11259, 458)
remove_features end (33851, 420) (11259, 420)
Database_column_compaction start (33851, 420)
Database_column_compaction end (33851, 399)
Database_column_compaction start (11259, 420)
Database_column_compaction end (11259, 399)
Framework_column_compaction start (33851, 399)
Framework_column_compaction end (33851, 387)
Framework_column_compaction start (11259, 399)
F

In [ ]:
train()

0:	learn: 42865.4670333	total: 335ms	remaining: 5m 35s
500:	learn: 19380.7599703	total: 2m 23s	remaining: 2m 22s
999:	learn: 18359.4244421	total: 4m 45s	remaining: 0us
0:	learn: 42864.3273149	total: 264ms	remaining: 4m 24s
500:	learn: 19380.9679693	total: 2m 18s	remaining: 2m 17s
999:	learn: 18420.2282819	total: 4m 36s	remaining: 0us
0:	learn: 42851.8414758	total: 256ms	remaining: 4m 15s
500:	learn: 19428.7606750	total: 2m 14s	remaining: 2m 14s
999:	learn: 18462.6561801	total: 4m 30s	remaining: 0us


In [ ]:

def predict():
  cols = ["","Country","Country_Count"]#
  emsamble = pd.read_csv(SUB_FILE)
  emsamble = emsamble.set_index('No')
  for d in cols:
    test_data = pd.read_csv("preprocess_"+TEST_FILE,index_col=0)
    if d in train_data.columns:
      del test_data[d]

    categorical_features_indices = []
    df = test_data.drop(TARGET, axis=1)
    for c in range(0,len(df.columns)):
      if 'float' not in str(df[df.columns[c]].dtype):
        categorical_features_indices.append(df.columns[c])
        test_data[df.columns[c]].fillna(-999,inplace=True)
        test_data[df.columns[c]] =  test_data[df.columns[c]].astype('category')
    
    model = CatBoostRegressor()
    model.load_model('cat_'+d)
    predictions = model.predict(test_data.drop(TARGET,axis=1).drop('weight',axis=1))
    print(len(predictions[predictions<0]))
    predictions[predictions<5000] = 5000

    submit = pd.read_csv(SUB_FILE)
    submit[TARGET] = predictions
    submit = submit.set_index('No')
    emsamble +=submit * (0.4/3)
    model = lgb.Booster(model_file='lgb_' + d)
    predictions = model.predict(test_data.drop(TARGET,axis=1).drop('weight',axis=1))
    print(len(predictions[predictions<0]))
    predictions[predictions<0] = 0

    submit = pd.read_csv(SUB_FILE)
    submit[TARGET] = predictions
    submit = submit.set_index('No')
    emsamble +=submit * (0.6/3)
    submit.to_csv('LGBM'+d+'.csv')
  
  emsamble.to_csv('Quevico_Submission.csv')
predict()